# 3.3 모델 구현하기

### 공통 처리

In [ ]:
# 공통 처리

# 불필요한 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 한글 글꼴 설정
import platform

if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='Apple Gothic')

# 데이터프레임 출력용 함수
from IPython.display import display

# 숫자 출력 조정
# 넘파이 부동소수점 출력 자리수 설정
np.set_printoptions(suppress=True, precision=4)

# 판다스 부동소수점 출력 자리수 설정
pd.options.display.float_format = '{:.4f}'.format

# 데이터프레임 모든 필드 출력
pd.set_option("display.max_columns",None)

# 그래프 글꼴 크기 설정
plt.rcParams["font.size"] = 14

# 난수 시드
random_seed = 123

### 유방암 데이터 집합

UCI 데이터 집합 배포 웹사이트: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

#### 분석 데이터 이미지
<img src="https://www.researchgate.net/profile/Nick_Street/publication/2512520/figure/fig2/AS:279373199495169@1443619169198/Snakes-After-Convergence-to-Cell-Nucleus-Boundaries-These-contours-are-the-nal.png" alt="Drawing" width="40%" align="left">


### 3.3.1 (1) 데이터 읽어 들이기

In [ ]:
# 유방암 데이터 집합 읽어 들이기

# 라이브러리 임포트
from sklearn.datasets import load_breast_cancer

# 데이터 내려받기
cancer = load_breast_cancer()

# 데이터에 대한 설명 읽기
print(cancer.DESCR)

In [ ]:
# 데이터프레임으로 변환하기

columns = [
    '반지름_평균', '텍스처_평균', '둘레길이_평균', '면적_평균',
    '평활도_평균', '콤팩트도_평균', '오목면_평균',
    '오목점_평균', '대칭성_평균', '프랙탈도_평균',
    '반지름_표준편차', '텍스처_표준편차', '둘레길이_표준편차',
    '면적_표준편차', '평활도_표준편차',
    '콤팩트도_표준편차', '오목면_표준편차', '오목점_표준편차',
    '대칭성_표준편차', '프랙탈도_표준편차',
    '반지름_최대', '텍스처_최대', '둘레길이_최대', '면적_최대',
    '평활도_최대', '콤팩트도_최대', '오목면_최대', '오목점_최대',
    '대칭성_최대', '프랙탈도_최대'
]

# 읽어 들인 데이터를 데이터프레임으로 변환
df = pd.DataFrame(cancer.data, columns=columns)

# 정답 데이터를 꺼냄
y = pd.Series(cancer.target)

### 3.3.2 (2) 데이터 확인

In [ ]:
# 입력 데이터를 화면에 출력하기

# 입력 데이터의 20번째 줄부터 24번째 줄까지 화면에 출력한다
display(df[20:25])

In [ ]:
# 정답 데이터를 화면에 출력하기

# 정답 데이터의 20번째 줄부터 24번째 줄까지 화면에 출력한다
print(y[20:25])

In [ ]:
# 데이터에 대한 통계 정보 확인하기

# 입력 데이터의 행과 열의 수를 확인
print(df.shape)
print()

# 정답 데이터의 1과 0의 건수 확인
print(y.value_counts())

In [ ]:
# 산포도를 그리기 위한 준비
# 입력 데이터를 대응하는 정답 데이터의 값에 따라 분할한다

# 정답 데이터 = 0(악성)인 데이터 골라내기
df0 = df[y==0]

# 정답 데이터 = 1(양성)인 데이터 골라내기
df1 = df[y==1]

display(df0.head())
display(df1.head())

In [ ]:
# 산포도 그리기

# 그래프의 크기를 설정
plt.figure(figsize=(6,6))

# 목적변수 값이 1인 데이터의 산포도 그리기
plt.scatter(df0['반지름_평균'], df0['텍스처_평균'], marker='x',
    c='b', label='악성')

# 목적변수 값이 1인 데이터의 산포도 그리기
plt.scatter(df1['반지름_평균'], df1['텍스처_평균'], marker='s',
    c='k', label='양성')

# 그리드 표시
plt.grid()

# 레이블 표시
plt.xlabel('반지름_평균')
plt.ylabel('텍스처_평균')

# 범례 그리기
plt.legend()

# 화면에 전체 그래프를 출력
plt.show()

### 3.3.3 (3) 데이터 전처리

In [ ]:
# 입력 데이터를 두 개의 필드로 줄이기

input_columns = ['반지름_평균', '텍스처_평균']
x = df[input_columns]
display(x.head())

### 3.3.4 (4) 데이터 분할

In [ ]:
# 학습 데이터와 검증 데이터 분할하기

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, 
    train_size=0.7, test_size=0.3, random_state=random_seed)

In [ ]:
# 분할된 데이터를 확인하기(요소 수)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# 분할된 데이터를 확인하기(요소 수)

display(x_train.head())
display(x_test.head())
display(y_train.head())
display(y_test.head())

### 3.3.5 (5) 알고리즘 선택하기

In [ ]:
# 알고리즘 선택

from sklearn.linear_model import LogisticRegression
algorithm = LogisticRegression(random_state=random_seed)

### 3.3.6 (6) 학습

In [ ]:
# 학습

algorithm.fit(x_train, y_train)
#print(algorithm)
print(algorithm.get_params())

### 3.3.7 (7) 예측

In [ ]:
# 예측

# predict 함수를 호출한다
y_pred = algorithm.predict(x_test)

# 예측 결과를 확인
print(y_pred)

### 3.3.8 (8) 평가

In [ ]:
# 정답 데이터와 예측 결과 비교하기

# 정답 데이터를 앞에서부터 10건 추려냄
# y_test는 데이터프레임이므로 value 속성에 담긴 넘파이 배열로 변환한다
y_test10 = y_test[:10].values
print(y_test10)

# 예측 결과를 앞에서부터 10건 추려냄
y_pred10 = y_pred[:10]
print(y_pred10)

In [ ]:
# 정답을 맞힌 건수 세기

# 정답 데이터 == 예측 결과　
w1 = (y_test10 == y_pred10)
print(w1)

# 정답을 맞힌 건수
w2 = w1.sum()
print(w2)

In [ ]:
# 정확도 계산하기

# 정답을 맞힌 건수를 계산
w = (y_test.values == y_pred)
correct = w.sum()

# 검증 데이터 전체 건수
N = len(w)

# 정확도 = (정답 수) / (검증 데이터 전체 건수)
score = correct / N

# 결과를 출력
print(f'정확도: {score:.04f}')

In [ ]:
# score 함수를 사용

# score 함수를 사용해도 정확도를 계산할 수 있다
score = algorithm.score(x_test, y_test)
print(f'score: {score:.04f}')

### 3.3.9 (9) 튜닝

In [ ]:
# 모델의 정확도 개선하기

# 원래 있던 30개 필드를 모두 포함해 학습 데이터와 검증 데이터를 다시 만든다
x2_train, x2_test, y_train, y_test = train_test_split(df, y, 
    train_size=0.7, test_size=0.3, random_state=random_seed)

# 로지스틱 회귀 모델도 다시 만든다
algorithm2 = LogisticRegression(random_state=random_seed)

# 새로 만든 학습 데이터로 학습을 진행한다
algorithm2.fit(x2_train, y_train)

# 검증 데이터로 정확도를 확인한다
score2 = algorithm2.score(x2_test, y_test)
print(f'score: {score2:.04f}')

## (보충설명) 결정경계 그리기
이 아래의 셀은 결정경계를 그리기 위한 코드와, 해당 구현에 대한 설명을 담고 있다.

상당히 어려운 내용이므로, 파이썬에 익숙하지 않은 독자는 건너뛰어도 좋다.

파이썬 구현에 관심있는 독자라면 도움이 될 것이다.

### 로지스틱 회귀의 내부 구현

로지스틱 회귀 모델은 내부적으로 다음과 같이 동작한다.

(1) 입력변수를 1차 함수에 입력해 실수값을 구한다.

(2) (1)에서 얻은 1차 함수의 함숫값을 시그모이드 함수에 입력해 확률값을 계산한다.

(3) 예측결과는 (2)에서 얻은 확률값이 0.5보다 크면 1, 작으면 0이 된다.

(1)에서 사용된 1차 함수의 기울기와 절편을 변수 coef_, intercept_로 구할 수 있다.
다음 코드는 이러한 성질을 이용해 내부변수 값을 사용한다.

그리고 이 모델은 다중분류를 위해 변수의 내부 변수를 저장할 수 있도록 했다.
그러므로 2차원 배열이 사용됐지만, 여기서는 이진분류를 적용하므로
첫 번째 요소([0])만이 사용된다.

### 내부변수의 값에 접근하기

In [ ]:
# 모델의 내부변수(절편, 계수)값 구하기

# x1, x2의 계수
w1 = algorithm.coef_[0][0]
w2 = algorithm.coef_[0][1]

# 절편
w0 = algorithm.intercept_[0]

# 값 확인하기
print(f'w0 = {w0:.4f}  w1 = {w1:.4f}  w2 = {w2:.4f}')

### boundary 함수의 정의

위에서 얻은 w0, w1, w2의 값을 사용해 산포도에 결정경계를 그리는 boundary 함수를 정의할 수 있다. 함수의 구현과 이를 유도한 식을 다음 셀에 싣는다.

In [ ]:
# 결정경계를 계산하는 함수

# 결정경계 계산 함수
# 0 = w0 + w1 * x + w2 * y를 y에 대해 풀면
# y = -(w0 + w1 * x)/ w2가 된다

def boundary(algorithm, x):
    w1 = algorithm.coef_[0][0]
    w2 = algorithm.coef_[0][1]
    w0 = algorithm.intercept_[0]
    y = -(w0 + w1 * x)/w2
    return y

### 결정경계의 경곗값 계산하기

다음 코드는 위에서 정의한 boundary 함수를 사용해 결정경계를 구성하는 경곗값의 좌표를 구하는 코드다.
이와 함께, 데이터가 담긴 데이터프레임에서 최솟값과 최댓값을 구해
그래프를 더 보기 좋게 그린다.

In [ ]:
# 결정경계의 경곗값 계산하기

# 결정경계의 x축 양끝 좌표값
x_range = np.array((df['반지름_평균'].min(), df['반지름_평균'].max()))

# 결정경계의 y축 양끝 좌표값
y_range = boundary(algorithm, x_range)

# y의 최솟값과 최댓값은 산포도의 점으로 결정한다
y_lim = (df['텍스처_평균'].min(), df['텍스처_평균'].max())

# 결과 확인
print('x축 양끝 좌표: ', x_range)
print('y축 양끝 좌표: ', y_range)
print('그래프의 y구간: ', y_lim)

### 산포도와 결정경계 그리기

모든 준비가 끝났다.
다음 코드는 코드 3-8에서 그린 산포도에 지금 계산한 결정경계를 겹쳐 그리는 코드다.

In [ ]:
# 산포도와 결정경계 그리기

# 그래프 크기 정의
plt.figure(figsize=(6,6))

# 목적변수의 값이 0인 데이터의 산포도
plt.scatter(df0['반지름_평균'], df0['텍스처_평균'], marker='x', c='b', label='악성')

# 목적변수의 값이 1인 데이터의 산포도
plt.scatter(df1['반지름_평균'], df1['텍스처_평균'], marker='s', c='k', label='양성')

# 결정경계
plt.plot(x_range, y_range, c='b', label='결정경계')

# 범위 지정
plt.ylim(y_lim)

# 레이블 추가
plt.xlabel('반지름_평균')
plt.ylabel('텍스처_평균')

# 범례 표시
plt.legend()

# 그리드 표시
plt.grid()

# 그래프 화면 출력
plt.show()